In [1]:
import sys
sys.path.insert(0, "../../src")
from pathlib import Path
from collections import Counter

import numpy as np
from scipy.special import softmax

from gen.util import read_data, write_jsonl
from rte.aggregate import generate_micro_macro_df

In [2]:
root_data = Path("../../data").resolve()
root_model = Path("../../models").resolve()

In [3]:
# constants
LOOKUP = {
    "verifiable": {"no": "NOT VERIFIABLE", "yes": "VERIFIABLE"},
    "label": {"nei": "NOT ENOUGH INFO", "r": "REFUTES", "s": "SUPPORTS"}
}

SEED = 123456789

LABEL2ID = {"SUPPORTS": 0, "NOT ENOUGH INFO": 1, "REFUTES": 2}
ID2LABEL = {0: "SUPPORTS", 1: "NOT ENOUGH INFO", 2: "REFUTES"}

# Init

In [4]:
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    TextClassificationPipeline,
    pipeline
)
import torch
torch.backends.cuda.matmul.allow_tf32 = True

# Huggingface Init

## Model

In [5]:
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

In [6]:
model_checkpoint = "xlnet-base-cased"
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, 
        num_labels=3, 
        id2label=ID2LABEL, 
        label2id=LABEL2ID
    )

model = model_init()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, do_lower_case=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess(examples):
    return tokenizer(examples["evidence"], examples["claim"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    results = {}
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="macro"))
    
    return results

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [8]:
dataset = ["fever", "climatefeverpure", "fever-climatefeverpure", "climatefever", "fever-climatefever"]
doc_sent = ["doc", "sent"]

di = 0
ds = 1

model_store_path = root_model.joinpath(model_checkpoint)
model_store_path.mkdir(exist_ok=True)
model_store_path = model_store_path / f"{dataset[di]}-{model_checkpoint}-{doc_sent[ds]}"

## Dataset

In [10]:
datap = root_data / f"{doc_sent[ds]}-dataset"

data = DatasetDict({
    "train": Dataset.from_list(read_data(datap / f"{dataset[di]}.train.n5.jsonl")),
    "validation": Dataset.from_list(read_data(datap / f"{dataset[di]}.dev.n5.jsonl")),
    "test": Dataset.from_list(read_data(datap / f"{dataset[di]}.test.n5.jsonl"))
}).map(preprocess, batched=True)

Map:   0%|          | 0/253140 [00:00<?, ? examples/s]

Map:   0%|          | 0/19108 [00:00<?, ? examples/s]

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

## Trainer

In [8]:
batch_size = 16
learning_rate = 4e-4
epoch = 4
metric_name = "f1"
warmup_ratio=0.1
save_steps=200
eval_steps=200

# Hyperparameter tuning

In [9]:
# shard the data if the dataset is large for hyperparameter tuning
shard = data["train"].num_rows > 50000
hp_tune_train = data["train"].shuffle(seed=SEED).shard(num_shards=5, index=1)

In [10]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=eval_steps,
    save_strategy = "no",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard"
)

_ = model.train()
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=hp_tune_train if shard else data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [11]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [1e-5, 3e-5, 2e-5]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
    }

def compute_objective(metrics):
    return metrics["f1"]

In [12]:
best_run = trainer.hyperparameter_search(
    direction="maximize", 
    n_trials=20, 
    hp_space=optuna_hp_space
)

[I 2023-07-03 12:57:58,567] A new study created in memory with name: no-name-2bef115b-c1dc-4b25-97b1-f3755592b842
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summar

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,1.104038,0.425536,0.439966,0.367711,0.336744
400,No log,0.873155,0.573349,0.563443,0.604527,0.463396
600,0.927000,0.573731,0.729216,0.708241,0.795350,0.659866
800,0.927000,0.626781,0.793644,0.785391,0.834284,0.770983
1000,0.435300,0.340676,0.878082,0.868632,0.877650,0.866576
1200,0.435300,0.298316,0.898464,0.890141,0.893090,0.890772
1400,0.435300,0.472975,0.860055,0.852273,0.873206,0.846291
1600,0.307100,0.300246,0.903544,0.899288,0.897161,0.897288
1800,0.307100,0.318785,0.901004,0.892937,0.901905,0.891543
2000,0.281900,0.320226,0.895986,0.897514,0.891704,0.892000


[I 2023-07-03 14:00:36,014] Trial 0 finished with value: 3.7200750259754667 and parameters: {'learning_rate': 1e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 5}. Best is trial 0 with value: 3.7200750259754667.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are new

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.628372,0.700161,0.682135,0.730822,0.628490
400,No log,0.412479,0.851134,0.850169,0.858349,0.843399
600,0.586500,0.291437,0.896419,0.892769,0.892628,0.889661
800,0.586500,0.358873,0.886136,0.886796,0.884524,0.880672
1000,0.257000,0.297880,0.901561,0.901587,0.898697,0.896829
1200,0.257000,0.256079,0.918597,0.916510,0.912997,0.913521
1400,0.257000,0.212030,0.929377,0.924447,0.925061,0.923925
1600,0.208700,0.275353,0.928695,0.925179,0.923107,0.923539
1800,0.208700,0.261209,0.924545,0.922519,0.919569,0.919763
2000,0.161000,0.265897,0.928819,0.926188,0.924124,0.924029


[I 2023-07-03 14:27:03,281] Trial 1 finished with value: 3.741136595545795 and parameters: {'learning_rate': 2e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 3}. Best is trial 1 with value: 3.741136595545795.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly

...

[I 2023-07-03 18:18:43,816] Trial 17 pruned. 
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stre

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.628372,0.700161,0.682135,0.730822,0.628490
400,No log,0.412479,0.851134,0.850169,0.858349,0.843399
600,0.586500,0.291437,0.896419,0.892769,0.892628,0.889661
800,0.586500,0.358873,0.886136,0.886796,0.884524,0.880672
1000,0.257000,0.297880,0.901561,0.901587,0.898697,0.896829
1200,0.257000,0.256079,0.918597,0.916510,0.912997,0.913521
1400,0.257000,0.212030,0.929377,0.924447,0.925061,0.923925
1600,0.208700,0.275353,0.928695,0.925179,0.923107,0.923539
1800,0.208700,0.261209,0.924545,0.922519,0.919569,0.919763
2000,0.161000,0.265897,0.928819,0.926188,0.924124,0.924029


[I 2023-07-03 18:45:24,063] Trial 18 finished with value: 3.741136595545795 and parameters: {'learning_rate': 2e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 3}. Best is trial 1 with value: 3.741136595545795.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newl

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.628372,0.700161,0.682135,0.730822,0.628490
400,No log,0.412479,0.851134,0.850169,0.858349,0.843399
600,0.586500,0.291437,0.896419,0.892769,0.892628,0.889661
800,0.586500,0.358873,0.886136,0.886796,0.884524,0.880672
1000,0.257000,0.297880,0.901561,0.901587,0.898697,0.896829
1200,0.257000,0.256079,0.918597,0.916510,0.912997,0.913521
1400,0.257000,0.212030,0.929377,0.924447,0.925061,0.923925
1600,0.208700,0.275353,0.928695,0.925179,0.923107,0.923539
1800,0.208700,0.261209,0.924545,0.922519,0.919569,0.919763
2000,0.161000,0.265897,0.928819,0.926188,0.924124,0.924029


[I 2023-07-03 19:12:04,617] Trial 19 finished with value: 3.741136595545795 and parameters: {'learning_rate': 2e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 3}. Best is trial 1 with value: 3.741136595545795.


In [13]:
best_run

BestRun(run_id='1', objective=3.741136595545795, hyperparameters={'learning_rate': 2e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 3}, run_summary=None)

## Train with best hyperparameters

In [ ]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=5,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard",
    tf32=True
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

for n, v in best_run.hyperparameters.items():
    if n == "per_device_train_batch_size" and v == 32:
        # out-of-memory not caught in hyperparameter tuning but
        # when whole data was used, OOM was thrown
        setattr(trainer.args, "per_device_train_batch_size", 4)
        setattr(trainer.args, "gradient_accumulation_steps", 8)
        setattr(trainer.args, "per_device_eval_batch_size", v)
    else:
        setattr(trainer.args, n, v)
    
trainer.train()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1000,0.270700,0.234412,0.923444,0.920040,0.921062,0.919548
2000,0.188700,0.273658,0.921662,0.921116,0.921194,0.919188
3000,0.168400,0.277822,0.920455,0.920125,0.921347,0.918102
4000,0.144800,0.185855,0.946204,0.944778,0.943507,0.943852
5000,0.148800,0.184193,0.946664,0.945569,0.943931,0.944541
6000,0.129400,0.173039,0.953043,0.951407,0.951149,0.950819
7000,0.130900,0.192036,0.945342,0.944658,0.943454,0.943355
8000,0.112100,0.171024,0.956032,0.954899,0.953967,0.954170
9000,0.097100,0.165694,0.959595,0.957972,0.957484,0.957601
10000,0.092800,0.149205,0.961090,0.959220,0.959301,0.959068


In [29]:
trainer.save_model(model_store_path.parent / (model_store_path.stem + ".out"))

# Evaluate

In [20]:
import pandas as pd
from sklearn.metrics import classification_report

## Test on validation data

In [21]:
preds = trainer.predict(data["validation"])
micro_val, macro_val = generate_micro_macro_df(data["validation"], preds)

In [22]:
print(classification_report(y_true=micro_val["actual"], y_pred=micro_val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.99      0.98      0.98      6666
        REFUTES       0.95      0.94      0.95      5347
       SUPPORTS       0.94      0.97      0.95      5386

       accuracy                           0.96     17399
      macro avg       0.96      0.96      0.96     17399
   weighted avg       0.96      0.96      0.96     17399



## Test on test data

In [25]:
preds = trainer.predict(data["test"])
micro_val, macro_val = generate_micro_macro_df(data["test"], preds)

In [26]:
print(classification_report(y_true=micro_val["actual"], y_pred=micro_val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.99      0.98      0.98      6666
        REFUTES       0.94      0.91      0.93      5272
       SUPPORTS       0.92      0.96      0.94      5389

       accuracy                           0.95     17327
      macro avg       0.95      0.95      0.95     17327
   weighted avg       0.95      0.95      0.95     17327

